In [1]:
import networkx as nx
import pandas as pd
import os
import random
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.nn import init
from torch.autograd import Variable
import itertools
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

import numpy as np
import time
import random
from sklearn.metrics import f1_score
from collections import defaultdict 
from dgl.nn import SAGEConv
import scipy.sparse as sp
import json
import seaborn as sns
import matplotlib.pyplot as plt

import dgl
from dgl.data.utils import save_graphs
from dgl import load_graphs

from dgl.nn import SAGEConv
import dgl.data

Using backend: pytorch


# Loads Graph

In [ ]:
#%%time
#G = bf.load_data("data/", "Spotify Playlist", 140, 300, 1000, False)

In [ ]:
%%time
dglG = load_graphs('./test_save.bin')[0][0]

In [ ]:
dglG

In [ ]:
for i in features.columns:
    dglG.ndata[i] = torch.tensor(features[i])

In [ ]:
dglG

# Edge Splitting

In [ ]:
# Split edge set for training and testing
u, v = dglG.edges()

In [ ]:
eids = np.arange(dglG.number_of_edges())


In [ ]:
eids = np.random.permutation(eids)


In [ ]:
test_size = int(len(eids) * 0.1)


In [ ]:
train_size = dglG.number_of_edges() - test_size


In [ ]:
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]


In [ ]:
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]


In [ ]:
# Find all negative edges and split them for training and testing


In [ ]:
adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))


In [ ]:
#adj_neg = 1 - adj.todense() - np.eye(dglG.number_of_nodes())


In [ ]:
#neg_u, neg_v = np.where(adj_neg != 0)


In [ ]:
#neg_eids = np.random.choice(len(neg_u), dglG.number_of_edges() // 2)


In [ ]:
#test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]


In [ ]:
#train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]

In [ ]:
train_g = dgl.remove_edges(dglG, eids[:test_size])

In [ ]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=dglG.number_of_nodes())
#train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=dglG.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=dglG.number_of_nodes())
#test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=dglG.number_of_nodes())

In [ ]:
class GraphSAGE(nn.Module):
    def __init__(self, in_feats, h_feats):
        super(GraphSAGE, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats, 'mean')
        self.conv2 = SAGEConv(h_feats, h_feats, 'mean')

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [ ]:
import dgl.function as fn

class DotPredictor(nn.Module):
    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            # Compute a new edge feature named 'score' by a dot-product between the
            # source node feature 'h' and destination node feature 'h'.
            g.apply_edges(fn.u_dot_v('h', 'h', 'score'))
            # u_dot_v returns a 1-element vector for each edge so you need to squeeze it.
            return g.edata['score'][:, 0]


In [ ]:
class MLPPredictor(nn.Module):
    def __init__(self, h_feats):
        super().__init__()
        self.W1 = nn.Linear(h_feats * 2, h_feats)
        self.W2 = nn.Linear(h_feats, 1)

    def apply_edges(self, edges):
        """
        Computes a scalar score for each edge of the given graph.

        Parameters
        ----------
        edges :
            Has three members ``src``, ``dst`` and ``data``, each of
            which is a dictionary representing the features of the
            source nodes, the destination nodes, and the edges
            themselves.

        Returns
        -------
        dict
            A dictionary of new edge features.
        """
        h = torch.cat([edges.src['h'], edges.dst['h']], 1)
        return {'score': self.W2(F.relu(self.W1(h))).squeeze(1)}

    def forward(self, g, h):
        with g.local_scope():
            g.ndata['h'] = h
            g.apply_edges(self.apply_edges)
            return g.edata['score']


In [ ]:
g.ndata['feat']

In [ ]:
model = GraphSAGE(features_arr.shape[1], 16)
# You can replace DotPredictor with MLPPredictor.
#pred = MLPPredictor(16)
pred = DotPredictor()

def compute_loss(pos_score):
    scores = ([pos_score])
    labels = (torch.ones(pos_score.shape[0]))
    return F.binary_cross_entropy_with_logits(scores, labels)

def compute_auc(pos_score):
    scores = torch.cat([pos_score]).numpy()
    labels = torch.cat(
        [torch.ones(pos_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)


In [ ]:
# ----------- 3. set up loss and optimizer -------------- #
# in this case, loss will in training loop
optimizer = torch.optim.Adam(itertools.chain(model.parameters(), pred.parameters()), lr=0.01)

# ----------- 4. training -------------------------------- #
all_logits = []
for e in range(100):
    # forward
    h = model(train_g, torch.tensor)
    pos_score = pred(train_pos_g, h)
    neg_score = pred(train_neg_g, h)
    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print('In epoch {}, loss: {}'.format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score
with torch.no_grad():
    pos_score = pred(test_pos_g, h)
    neg_score = pred(test_neg_g, h)
    print('AUC', compute_auc(pos_score, neg_score))

In [ ]:
#G = bf.load_data("data/", "Spotify Playlist", 140, 300, 1000, False)